# 라벨링된 채널 정보 -> 비디오 100개씩 추출 및 라벨링 적용

In [ ]:
from youtube_crawler import youtubeCrawler
import json
import pandas as pd

with open('APIkey.json') as f:
    key = json.load(f);
    apikey = key['APIkey'];


YC = youtubeCrawler(apikey)

channel_df = pd.read_csv('channel_labeled.csv')
channel_df

video_labeled_df = pd.DataFrame(columns=['video_id', 'difficulty', 'channel_id'])


for idx, channelID in enumerate(channel_df['channel_id']):
    difficulty = channel_df['difficulty'][idx]
    try:
        new_videos = YC.get_recent_videos(channelID)
        for video_id in new_videos:
            video_labeled_df.loc[len(video_labeled_df)] = [video_id, difficulty, channelID]
        
    except:
        print('something wrong')
        break
    print(f'{idx} {channelID} OK')

video_labeled_df.to_csv('video_labeled.csv', sep=',', index=False)

    

# 분석기를 통해 비디오 스크립트 분석 

 punctuator 가 머신러닝을 통해 작동하므로 적절한 환경에서 실행해야 오래 걸리지 않음

In [ ]:
import pandas as pd
from core import analyzeAll

import json

no_caption_videos =[]
train_data = pd.DataFrame(columns=['videoId','totalWords','totalUniqueWords','totalSentences','avgSyllPerSec','avgCEFRScore','readability','uncommonRatio','A1ratio','A2ratio','B1ratio','B2ratio','C1ratio','C2ratio','Nratio'])

video_labeled = pd.read_csv('video_labeled.csv')
for idx,video_id in enumerate(video_labeled['video_id']):
    if idx > 930:
        print(f'{idx} {video_id} start')
        try:
            result = analyzeAll(video_id)
            no_script_result = json.loads(result)
            del no_script_result['script']
            train_data.loc[len(train_data)] = no_script_result
        except NotImplementedError:
            print(f'{idx} {video_id} no captions')
            no_caption_videos.append(video_id)
        if idx % 300 == 0:
            train_data.to_csv('video_train_1.csv',sep=',',index=False)
            print("####temp download#########")
        


train_data.to_csv('video_train_1.csv',sep=',',index=False)
print(no_caption_videos)

# 난이도 라벨과 분석 결과 합치기

In [ ]:
import pandas as pd

data0 = pd.read_csv('video_train.csv', sep=',')
data1 = pd.read_csv('video_train_1.csv', sep=',')
data2 = pd.read_csv('video_train_2.csv', sep=',')
data3 = pd.read_csv('video_train_3.csv', sep=',')


merge = pd.concat([data0,data1,data2,data3], ignore_index=True)

label = pd.read_csv('video_labeled.csv', sep=',')

result = pd.merge(merge, label, how='left', left_on='videoId', right_on='video_id')
result.drop(['channel_id', 'video_id'], axis=1, inplace=True)
result.to_csv('capstone_train_data.csv', sep=',', index=False)
